In [1]:
from datasets import Dataset
from time import perf_counter
from autofaiss import build_index
from tqdm import tqdm
import polars as pl
import numpy as np
from data_load import load, embed

In [2]:
df, col, path = load()
df.head()

Data Shape: (72110, 6)


id,q_item_id,q_level1_global_be_category,q_item_name,q_link_first_image,q_item_name_clean
u32,i64,str,str,str,str
0,17185182528,"""Men Shoes""","""Giày Sandal Nam VENTO HYBIRD d…","""https://cf.shopee.sg/file/3105…","""giày sandal nam vento hybird d…"
1,24128723386,"""Men Shoes""","""Sandal nhựa Thái Lan Red Apple…","""https://cf.shopee.sg/file/vn-1…","""sandal nhựa thái lan red apple…"
2,11286133785,"""Men Shoes""","""Sticker jibbitz gắn dép sục Cr…","""https://cf.shopee.sg/file/6c70…","""sticker jibbitz gắn dép sục cr…"
3,27000803082,"""Men Shoes""","""Túi 50 gói khăn ướt lau giày d…","""https://cf.shopee.sg/file/vn-1…","""túi 50 gói khăn ướt lau giày d…"
4,24915597347,"""Men Shoes""","""SKECHERS - Giày sneakers nam c…","""https://cf.shopee.sg/file/vn-1…","""skechers - giày sneakers nam c…"


In [3]:
item = df['q_item_name_clean'].to_list()
embeddings, file_embed, df = embed(df, path)

In [4]:
start = perf_counter()

file_index = str(path / 'ip.index')
build_index(
    str(file_embed.parent),
    index_path=file_index,
    index_infos_path=str(path / f'index.json'),
    save_on_disk=True,
    metric_type='ip',
    verbose=30,
)

end = perf_counter() - start
print(f'Total Index Time: {end:,.0f}s')

  0%|                                                                | 0/1 [00:00<?, ?it/s]


Total Index Time: 31s


In [5]:
dataset = Dataset.from_polars(df)
dataset.load_faiss_index('vector', file_index)

In [6]:
start = perf_counter()

total_sample = len(dataset)
batch_size = 20_000
num_batches = (total_sample + batch_size) // batch_size
print(f'Total batches: {num_batches}, Batch size: {batch_size:,.0f}')
for i, idx in tqdm(enumerate(range(num_batches), start=1), total=num_batches):
    start_idx = idx * batch_size
    end_idx = min(start_idx + batch_size, total_sample)
    batch = embeddings[start_idx:end_idx]

    score, result = dataset.get_nearest_examples_batch(
        'vector',
        batch,
        k=10,
    )
    
    df_result = pl.DataFrame(result)
    dict_ = {f'score': [list(np.round(arr, 6)) for arr in score]}
    df_score = pl.DataFrame(dict_)

end = perf_counter() - start
print(f'Total Query Time: {end:,.0f}s')

Total batches: 4, Batch size: 20,000


100%|████████████████████████████████████████████████████████| 4/4 [03:44<00:00, 56.21s/it]

Total Query Time: 225s


In [7]:
df_result

id,q_item_id,q_level1_global_be_category,q_item_name,q_link_first_image,q_item_name_clean,vector
list[i64],list[i64],list[str],list[str],list[str],list[str],list[list[f64]]
"[60000, 26822, … 44792]","[29956095623, 27051603542, … 29904291285]","[""Men Shoes"", ""Men Shoes"", … ""Men Shoes""]","[""（Full Box） Giày Thể Thao ấdidas^_^Originals Handball Spezial Dark Green 36-45"", ""Dép Lê Hm - Dép Hm Sừng Trâu Gabriel Dép HM Bản Cao Cấp Hàng Supper 2024"", … ""DÉP DA NAM QUAI NGANG DA NHÂN TẠO DẬP NHĂN ĐẾ EVA TOROSHOES SIÊU NHẸ MÃ TORO D22""]","[""https://cf.shopee.sg/file/sg-11134201-7rdxc-ly3e04hnwv3kea"", ""https://cf.shopee.sg/file/vn-11134207-7r98o-lwk6frvf4lw94f"", … ""https://cf.shopee.sg/file/vn-11134207-7r98o-lxngmn6504bd1d""]","[""（full box） giày thể thao ấdidas^_^originals handball spezial dark green 36-45"", ""dép lê hm - dép hm sừng trâu gabriel dép hm bản cao cấp hàng supper 2024"", … ""dép da nam quai ngang da nhân tạo dập nhăn đế eva toroshoes siêu nhẹ mã toro d22""]","[[0.019257, -0.001904, … 0.058044], [0.047272, -0.009453, … 0.03186], … [0.008972, -0.015282, … 0.063416]]"
"[60001, 41519, … 71940]","[29405929507, 27456095857, … 22044801222]","[""Men Shoes"", ""Men Shoes"", … ""Men Shoes""]","[""Giày thể thao onitsuka tiger tokuken các màu hot nhất thị trường da lộn êm chân đủ size nam nữ"", ""（Full Box） Giày Thể Thao ấdidas^_^Originals Handball Spezial Blue White 36-45"", … ""[FREE SHIP] Dép Nam Quai Ngang Đẹp, Dép ICON Chất Liệu Cao Su, Đế PU Chống Trơn Trượt Êm Chân Thoải Mái""]","[""https://cf.shopee.sg/file/vn-11134201-7r98o-ly0tez8binkpbf"", ""https://cf.shopee.sg/file/sg-11134201-7rdx3-ly3e5nua6r5fb4"", … ""https://cf.shopee.sg/file/vn-11134207-7qukw-lj8tjq2a1hjg0f""]","[""giày thể thao onitsuka tiger tokuken các màu hot nhất thị trường da lộn êm chân đủ size nam nữ"", ""（full box） giày thể thao ấdidas^_^originals handball spezial blue white 36-45"", … ""dép nam quai ngang đẹp, dép icon chất liệu cao su, đế pu chống trơn trượt êm chân thoải mái""]","[[0.003141, 0.020157, … -0.004375], [0.032532, 0.011391, … 0.000307], … [0.000973, 0.009018, … 0.004787]]"
"[60002, 39480, … 35341]","[24269366719, 1063630473, … 20019466082]","[""Men Shoes"", ""Men Shoes"", … ""Men Shoes""]","[""GIÀY CHUNKY DERBY DA CỘT DÂY- CHUOTCONG 7716D"", ""Giày sandal nhựa nam nữ"", … ""(Bảo hành 1 năm) Giày Thể Thao Nam Nữ Tăng Chiều Cao Nờ Y ĐẾ Nâu Em Eo Bi - Phiên Bản Cao Cấp""]","[""https://cf.shopee.sg/file/vn-11134207-7r98o-lsxr8h8ukzvd94"", ""https://cf.shopee.sg/file/b880de9da61a47d10d44bb1d2402c3de"", … ""https://cf.shopee.sg/file/5d90082237f7175aa45884bd462ea87e""]","[""giày chunky derby da cột dây- chuotcong 7716d"", ""giày sandal nhựa nam nữ"", … ""giày thể thao nam nữ tăng chiều cao nờ y đế nâu em eo bi - phiên bản cao cấp""]","[[-0.050354, 0.031235, … 0.017014], [-0.050262, 0.023941, … -0.017487], … [-0.034637, 0.03537, … -0.019882]]"
"[60003, 62649, … 3313]","[19389422644, 2974950900, … 24618850878]","[""Men Shoes"", ""Men Shoes"", … ""Men Shoes""]","[""Dép cặp nam nữ"", ""[XUẤT KHẨU_PHOM LỚN_SIÊU BỀN] Dép nhựa nam quai ngang chất lượng, êm chân SM004"", … ""Dép nam GC xanh đỏ quai ngang cao su non-dép GC siêu phẩm 2024""]","[""https://cf.shopee.sg/file/vn-11134207-7qukw-lhk4lt7rixmp0c"", ""https://cf.shopee.sg/file/vn-11134207-7r98o-lo548j8ps1yvf1"", … ""https://cf.shopee.sg/file/vn-11134207-7r98o-ly208ovhntgb45""]","[""dép cặp nam nữ"", ""dép nhựa nam quai ngang chất lượng, êm chân sm004"", … ""dép nam gc xanh đỏ quai ngang cao su non-dép gc siêu phẩm 2024""]","[[0.027557, -0.023743, … -0.029388], [0.027557, -0.023758, … -0.029526], … [0.032776, -0.023407, … -0.004501]]"
"[60004, 38525, … 48046]","[17438630484, 29053261141, … 24832663355]","[""Men Shoes"", ""Men Shoes"", … ""Men Shoes""]","[""Giày Derby Nam công sở tăng chiều cao da bò thật CK022"", ""Sneaker Nam - Giày thể thao nam thời trang chạy bộ thoáng khí nhiều màu sắc lựa chọn HKT457 Gnar Sneaker"", … ""Fashion Giày thể thao Adidas Samba Classics OG White Halo Blue